In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
weather = pd.read_csv('weather_features.csv')
print(weather.shape)
weather['dateTime'] = pd.to_datetime(weather['dt_iso'],utc=True)
weather = weather.set_index(pd.DatetimeIndex(weather['dateTime']))
weather = weather.drop(['dt_iso'],axis=1)
weather['temp'] = weather['temp']-273.15
weather['temp_max'] = weather['temp_max']-273.15
weather['temp_min'] = weather['temp_min']-273.15
weather["month"] = weather["dateTime"].dt.month
weather["season"] = weather["dateTime"].dt.quarter
weather["year"] = weather["dateTime"].dt.year
weather["day"] = weather["dateTime"].dt.weekday
print(weather.shape)

(178396, 17)
(178396, 21)


In [3]:
weather.head()

,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,...,clouds_all,weather_id,weather_main,weather_description,weather_icon,dateTime,month,season,year,day
dateTime,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,Valencia,-2.675,-2.675,-2.675,1001,77,1,62,0.0,0.0,...,0,800,clear,sky is clear,01n,2014-12-31 23:00:00+00:00,12,4,2014,2
2015-01-01 00:00:00+00:00,Valencia,-2.675,-2.675,-2.675,1001,77,1,62,0.0,0.0,...,0,800,clear,sky is clear,01n,2015-01-01 00:00:00+00:00,1,1,2015,3
2015-01-01 01:00:00+00:00,Valencia,-3.464,-3.464,-3.464,1002,78,0,23,0.0,0.0,...,0,800,clear,sky is clear,01n,2015-01-01 01:00:00+00:00,1,1,2015,3
2015-01-01 02:00:00+00:00,Valencia,-3.464,-3.464,-3.464,1002,78,0,23,0.0,0.0,...,0,800,clear,sky is clear,01n,2015-01-01 02:00:00+00:00,1,1,2015,3
2015-01-01 03:00:00+00:00,Valencia,-3.464,-3.464,-3.464,1002,78,0,23,0.0,0.0,...,0,800,clear,sky is clear,01n,2015-01-01 03:00:00+00:00,1,1,2015,3


In [4]:
weather['weather_main'].unique()

array(['clear', 'clouds', 'rain', 'mist', 'thunderstorm', 'drizzle',
       'fog', 'smoke', 'haze', 'snow', 'dust', 'squall'], dtype=object)

In [5]:
energy = pd.read_csv('energy_dataset.csv')
print(energy.shape)
energy['dateTime'] = pd.to_datetime(energy['time'],utc=True)
energy = energy.set_index(pd.DatetimeIndex(energy['dateTime']))
energy = energy.drop(['time'],axis=1)
energy["month"] = energy["dateTime"].dt.month
energy["season"] = energy["dateTime"].dt.quarter
energy["year"] = energy["dateTime"].dt.year
energy["day"] = energy["dateTime"].dt.weekday


(35064, 29)


In [36]:
x = weather[['temp','temp_max','temp_min','wind_speed','pressure','wind_deg','humidity','month','season','year','day']].resample('D').mean()
y = energy[['price actual','total load actual','month','season','year','day']].resample('D').mean()

x = x[(x['day']!=5) & (x['day']!=6)]
y = y[(y['day']!=5) & (y['day']!=6)]

In [96]:
x[['temp','wind_speed','pressure','wind_deg']].tail(10)

,temp,wind_speed,pressure,wind_deg
dateTime,,,,
2018-12-18 00:00:00+00:00,7.788583,2.650000,1022.766667,179.366667
2018-12-19 00:00:00+00:00,10.678240,2.976000,1021.416000,199.056000
2018-12-20 00:00:00+00:00,9.670410,2.106557,1025.639344,194.213115
2018-12-21 00:00:00+00:00,9.986606,3.039370,1029.472441,184.937008
2018-12-24 00:00:00+00:00,8.546364,1.892562,1027.933884,155.809917
2018-12-25 00:00:00+00:00,8.068810,1.515873,1028.000000,149.968254
2018-12-26 00:00:00+00:00,8.127561,1.658537,1028.414634,167.699187
2018-12-27 00:00:00+00:00,7.521707,1.512195,1027.170732,174.349593
2018-12-28 00:00:00+00:00,8.003821,1.951220,1028.081301,176.365854


In [64]:
x.corr()

,temp,temp_max,temp_min,wind_speed,pressure,wind_deg,humidity,month,season,year,day
temp,1.000000,0.987932,0.988223,-0.111838,-0.042379,-0.143932,-0.626419,0.245475,0.255456,-0.015661,-0.004027
temp_max,0.987932,1.000000,0.953621,-0.104295,-0.044952,-0.149369,-0.626420,0.255947,0.262123,-0.053199,-0.005582
temp_min,0.988223,0.953621,1.000000,-0.121490,-0.039096,-0.140711,-0.610767,0.229138,0.243832,0.020803,-0.003202
wind_speed,-0.111838,-0.104295,-0.121490,1.000000,-0.022126,0.518528,-0.094265,-0.312938,-0.323449,0.000607,-0.028748
pressure,-0.042379,-0.044952,-0.039096,-0.022126,1.000000,-0.015560,0.002261,-0.035478,-0.037337,-0.039480,0.043996
wind_deg,-0.143932,-0.149369,-0.140711,0.518528,-0.015560,1.000000,-0.016630,-0.236662,-0.262333,-0.087934,0.025861
humidity,-0.626419,-0.626420,-0.610767,-0.094265,0.002261,-0.016630,1.000000,0.045821,0.059968,0.006326,0.031065
month,0.245475,0.255947,0.229138,-0.312938,-0.035478,-0.236662,0.045821,1.000000,0.971510,-0.008041,0.003236
season,0.255456,0.262123,0.243832,-0.323449,-0.037337,-0.262333,0.059968,0.971510,1.000000,-0.005336,-0.000307
year,-0.015661,-0.053199,0.020803,0.000607,-0.039480,-0.087934,0.006326,-0.008041,-0.005336,1.000000,-0.003322


## Baseline model 

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [147]:
from sklearn.linear_model import LinearRegression,RidgeCV, Ridge, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
X = x[['temp','pressure','wind_deg','wind_speed']].values.reshape(-1,4)
Y = y['price actual'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1234)
reg = LinearRegression().fit(X_train, y_train)

print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, reg.predict(X_train)), 2)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, reg.predict(X_test)), 2)))

print("\nR_squared: "+ str(np.round(reg.score(X_train, y_train),2)))



print("\nCoefficients: " + str(np.round(reg.coef_,2)))


Training MSE: 108.01

Test MSE:    108.42

R_squared: 0.16

Coefficients: [[-0.13  0.   -0.01 -4.48]]


In [148]:

alphas = np.linspace(0.01,1000,2000)

ridgecv = RidgeCV(alphas=alphas, scoring = 'neg_mean_squared_error')
ridgecv.fit(X_train, y_train)
print(ridgecv.alpha_)
ridge_base = Ridge(alpha = ridgecv.alpha_).fit(X_train, y_train)
mean_squared_error(y_test, ridge_base.predict(X_test))
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, ridge_base.predict(X_train)), 2)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, ridge_base.predict(X_test)), 2)))

print("\nR_squared: "+ str(ridge_base.score(X_train, y_train)))



print("\nCoefficients: " + str(ridge_base.coef_))

1000.0

Training MSE: 114.07

Test MSE:    111.9

R_squared: 0.10971869797903011

Coefficients: [[-1.15861381e-01  3.82718307e-04 -5.45109662e-02 -1.47452382e+00]]






## Season: Winter

# Linear regression

In [153]:
X = x[x['season']==1][['temp','pressure','wind_deg','wind_speed']].values.reshape(-1,4)
Y = y[y['season']==1]['price actual'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1234)

reg = LinearRegression().fit(X_train, y_train)


print('Winter')
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, reg.predict(X_train)), 2)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, reg.predict(X_test)), 2)))

print("\nR_squared: "+ str(np.round(reg.score(X_train, y_train),2)))



print("\nCoefficients: " + str(np.round(reg.coef_,2)))

Winter

Training MSE: 108.28

Test MSE:    117.68

R_squared: 0.39

Coefficients: [[-2.    0.   -0.04 -3.1 ]]


## Ridge regression

In [129]:

ridgecvW = RidgeCV(alphas=alphas, scoring = 'neg_mean_squared_error')
ridgecvW.fit(X_trainW, y_trainW)
print(ridgecvW.alpha_)
ridge_winter = Ridge(alpha = ridgecvW.alpha_).fit(X_trainW, y_trainW)

print('\nTraining MSE: ' + str(round(mean_squared_error(y_trainW, ridge_winter.predict(X_trainW)), 2)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_testW, ridge_winter.predict(X_testW)), 2)))

print("R_squared")
print(ridge_winter.score(X_trainW, y_trainW))

print('coefficients')
print(ridge_winter.coef_)

576.2923811905953

Training MSE: 116.98

Test MSE:    136.82
R_squared
0.34498157752230985
coefficients
[[-1.56166095e+00  2.80152027e-04 -6.83242488e-02 -9.65562990e-01]]


## LASSO

In [72]:
lasso = Lasso(max_iter = 10000)
lassocv = LassoCV(alphas = alphas, cv = 10, max_iter = 100000)
lassocv.fit(X_train, y_train)

lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, lasso.predict(X_train)), 0)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, lasso.predict(X_test)), 0)))

print(lasso.coef_)

C:\Users\antti\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training MSE: 121.0

Test MSE:    143.0
[-1.70540108e+00  3.00810930e-04 -7.48674262e-02 -2.17126563e-01]


## Season: Spring

In [154]:
X = x[x['season']==2][['temp','pressure','wind_deg','wind_speed']].values.reshape(-1,4)
Y = y[y['season']==2]['price actual'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1234)

reg = LinearRegression().fit(X_train, y_train)


print('Spring')
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, reg.predict(X_train)), 2)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, reg.predict(X_test)), 2)))

print("\nR_squared: "+ str(np.round(reg.score(X_train, y_train),2)))



print("\nCoefficients: " + str(np.round(reg.coef_,2)))

Spring

Training MSE: 86.44

Test MSE:    95.63

R_squared: 0.19

Coefficients: [[ 0.6  -0.43 -0.02 -4.68]]


## Season: Summer

In [155]:
X = x[x['season']==3][['temp','pressure','wind_deg','wind_speed']].values.reshape(-1,4)
Y = y[y['season']==3]['price actual'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1234)

reg = LinearRegression().fit(X_train, y_train)


print('Summer')
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, reg.predict(X_train)), 2)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, reg.predict(X_test)), 2)))

print("\nR_squared: "+ str(np.round(reg.score(X_train, y_train),2)))



print("\nCoefficients: " + str(np.round(reg.coef_,2)))

Summer

Training MSE: 89.31

Test MSE:    110.71

R_squared: 0.06

Coefficients: [[ 0.78  0.07 -0.05  1.71]]


## Season: Autumn

In [156]:
X = x[x['season']==4][['temp','pressure','wind_deg','wind_speed']].values.reshape(-1,4)
Y = y[y['season']==4]['price actual'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1234)

reg = LinearRegression().fit(X_train, y_train)

print('Autumn')

print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, reg.predict(X_train)), 2)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, reg.predict(X_test)), 2)))

print("\nR_squared: "+ str(np.round(reg.score(X_train, y_train),2)))



print("\nCoefficients: " + str(np.round(reg.coef_,2)))

Autumn

Training MSE: 24.81

Test MSE:    23.84

R_squared: 0.22

Coefficients: [[-0.41  0.01 -0.03 -1.67]]


In [175]:
x2 = x[(x['day']!=5) & (x['day']!=6)]
y2 = y[(y['day']!=5) & (y['day']!=6)]
x2 = pd.get_dummies(x2, columns=["season"], prefix=["season"],drop_first=True)
X = x2[['temp','pressure','wind_deg','wind_speed','humidity','season_2.0','season_3.0','season_4.0']].values.reshape(-1,8)
Y = y2['price actual'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1234)

In [183]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

x2 = x[(x['day']!=5) & (x['day']!=6)]
y2 = y[(y['day']!=5) & (y['day']!=6)]
x2 = pd.get_dummies(x2, columns=["season"], prefix=["season"],drop_first=True)
X = x2[['temp','pressure','wind_deg','wind_speed','humidity','season_2.0','season_3.0','season_4.0']].values.reshape(-1,8)
Y = y2['price actual'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1234)


model = Sequential()


model.add(Dense(10,input_dim = 8, activation="relu"))

 
model.add(Dense(10, activation="relu")) 

model.add(Dense(1, activation='relu'))



opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)


model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])

model.fit(X_train, y_train, epochs=2000,  validation_data=(X_test, y_test))

Epoch 1/2000
23/23 [==============================] - 0s 5ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 2/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 3/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 4/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 5/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 6/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 7/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 37

23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 58/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 59/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 60/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 61/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 62/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 63/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033

23/23 [==============================] - 0s 1ms/step - loss: 3702.0757 - mse: 3702.0757 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 114/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 115/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 116/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 117/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 118/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 119/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 373

23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 170/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 171/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 172/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 173/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 174/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 175/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 373

23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 226/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 227/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 228/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 229/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 230/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 231/2000
23/23 [==============================] - 0s 4ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 373

...

23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1733/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1734/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0757 - mse: 3702.0757 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1735/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1736/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1737/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1738/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_ms

Epoch 1788/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1789/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1790/2000
23/23 [==============================] - 0s 2ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1791/2000
23/23 [==============================] - 0s 2ms/step - loss: 3702.0757 - mse: 3702.0757 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1792/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1793/2000
23/23 [==============================] - 0s 2ms/step - loss: 3702.0757 - mse: 3702.0757 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1794/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 37

Epoch 1844/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1845/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1846/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1847/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1848/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1849/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1850/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 37

Epoch 1900/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1901/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1902/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1903/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1904/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1905/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1906/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 37

Epoch 1956/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1957/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1958/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1959/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0750 - mse: 3702.0750 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1960/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1961/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0747 - mse: 3702.0747 - val_loss: 3730.9033 - val_mse: 3730.9033
Epoch 1962/2000
23/23 [==============================] - 0s 1ms/step - loss: 3702.0754 - mse: 3702.0754 - val_loss: 37

In [141]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
X, y = load_diabetes(return_X_y=True)
clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X, y)
clf.score(X, y)

0.5166287840315842

In [145]:
y.shape


(442,)